In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re
import gensim
from gensim.models import KeyedVectors
import numpy as np
from collections import Counter
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases
from nltk import sent_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim.downloader as api
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer
from preprocessing import *

In [2]:
w2v_pretrained = Word2Vec.load("w2vPreTrained")

In [3]:
dataset = pd.read_csv("amazon_alexa.tsv", sep = "\t", encoding = "utf-8")
print(dataset.shape)
dataset.dropna(inplace = True)
print(dataset.shape)
dataset.drop(dataset[dataset.rating == 3].index, inplace=True)
print(dataset.shape)
dataset.drop_duplicates(subset = "verified_reviews", inplace = True)
print(dataset.shape)

(3150, 6)
(3150, 6)
(2998, 6)
(2196, 6)


In [4]:
X = np.array(dataset["verified_reviews"].values).reshape(-1, 1)
y = list(dataset["feedback"].values)

In [5]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state = 0)

X, y = undersampler.fit_resample(X, y)


print('Resampled dataset shape %s' % Counter(y))

Resampled dataset shape Counter({1: 412, 0: 206})


In [6]:
X_temp = []

for rev in X:
  X_temp.append(rev[0])

In [7]:
new_text, new_sent_tok = tokenize_list_of_text(X_temp, custom_stopwords, False, 2)

total number of types extracted is: 1788


In [8]:
cleaned_reviews = frequency_cleaning(new_sent_tok, 2)

In [9]:
bigrams = Phrases(cleaned_reviews, scoring="npmi", threshold=0.60) #estrae le collocazioni tramite PMI
bigrams[cleaned_reviews][0]

['like',
 'fact',
 'answer',
 'not_see',
 'real',
 'need',
 'household',
 'though',
 'good',
 'day',
 'deal']

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(bigrams[cleaned_reviews], y, test_size=0.20, random_state=10)

In [11]:
negative_reviews = []
for rev, label in zip(X_train, Y_train):
    if label == 0:
        negative_reviews.append(rev)
    
generated_reviews = generate_samples(negative_reviews, int(len(negative_reviews)/2), w2v_pretrained)

# run this only one time
X_train.extend(generated_reviews)
Y_train.extend([0 for x in generated_reviews])

['joke']
-
['laugh']
--------------------------
['price', 'product', 'nice', 'quality', 'nice', 'feature', 'definitely', 'reason', 'give', 'think', 'may', 'buyer', 'error', 'first', 'ignore', 'product', 'plug', 'time', 'work', 'really', 'unlike', 'not_a', 'stand', 'device', 'also', 'speaker', 'not_very', 'loud', 'buy', 'bluetooth', 'speaker', 'sure', 'lot', 'figure', 'use', 'kind', 'seem', 'like', 'device', 'plus', 'set', 'awful', 'would', 'definitely', 'money', 'buy', 'one', 'actually']
-
['cost', 'product', 'decent', 'quality', 'decent', 'feature', 'definitely', 'reason', 'present', 'imagine', 'may', 'buyer', 'error', 'start', 'ignore', 'product', 'plug', 'sentence', 'run', 'truly', 'different', 'not_a', 'base', 'device', 'also', 'speaker', 'not_very', 'loudly', 'purchase', 'bluetooth', 'speaker', 'sure', 'plenty', 'figure', 'utilize', 'kind', 'seem', 'wish', 'device', 'plus', 'adjust', 'terrible', 'would', 'definitely', 'money', 'purchase', 'one', 'really']
-------------------------

In [12]:
tfidf = TfidfVectorizer(tokenizer=lambda i:i, lowercase=False, min_df = 0)
tfidf_model = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)
review = 0
for score, feature in zip(tfidf_model.toarray()[review], tfidf.get_feature_names_out()):
    if score > 0.0:
        print(feature, score)

box 0.2841244562320641
even_though 0.27777187965605415
far 0.2128361323959227
happy 0.5158776085523762
like 0.16414035678091646
look 0.2248712823584504
new 0.21466459777606126
original 0.26694879335690763
perfectly 0.29964046313926207
purchase 0.20300269887524253
refurbish 0.22956420296650182
two 0.237471197305058
week 0.27208930626953637
work 0.14736371838790915


In [13]:
w2v_model = Word2Vec(X_train, vector_size=100, window = 10, min_count = 0, sg=1, hs = 1, seed = 5, epochs=100)

In [14]:
w2v_model.wv.most_similar("love", topn = 10)

[('great', 0.4427104592323303),
 ('use', 0.41806232929229736),
 ('cook', 0.41679319739341736),
 ('trailer', 0.3842005133628845),
 ('list', 0.3723629117012024),
 ('movie', 0.3719281852245331),
 ('find', 0.36989104747772217),
 ('want', 0.3640190362930298),
 ('favorite', 0.3593107759952545),
 ('live', 0.3583296835422516)]

In [15]:
w2v_model.wv.most_similar("love", topn = 2000)[-10:]

[('unplug', -0.060755953192710876),
 ('money', -0.0664283037185669),
 ('listen', -0.06782251596450806),
 ('phone', -0.07216014713048935),
 ('zigbee', -0.0758974477648735),
 ('useful', -0.0774853527545929),
 ('half', -0.07899769395589828),
 ('return', -0.08647407591342926),
 ('ignore', -0.11667533963918686),
 ('also', -0.1351698786020279)]

In [16]:
def review_vectors(tokens, size = 300, weights = [], pretrained = False, pretrained_embeddings = None):
    
    """Genera un vettore per ogni recensione: questo vettore
    è calcolato come la media ponderata (t * w: token vettore * peso tfidf) dei vettori dei token nella recensione.
    Il vettore risultante è normalizzato alla fine."""

    vec = np.zeros(size).reshape((1, size))
    count = 0

    for word, weight in zip(tokens, weights):
        try:
            if pretrained:
                vec += pretrained_embeddings[word] 
                count +=1
            else:
                vec += w2v_model.wv[word] * weight
                count +=1
        except KeyError:
            # print("non trovo", word)
            continue

    if count!= 0:
        vec = vec / norm(vec)
        
    return vec

In [17]:
w2v_X_train = np.zeros((len(X_train), w2v_model.vector_size))
for i in range(len(X_train)):
    w2v_X_train[i,:] = review_vectors(tfidf.inverse_transform(tfidf_model[i, :])[0], w2v_model.vector_size, tfidf_model[i,:].data, False)
w2v_X_train.shape

(570, 100)

In [18]:
w2v_X_test = np.zeros((len(X_test), w2v_model.vector_size))
for i in range(len(X_test)):
    w2v_X_test[i,:] = review_vectors(tfidf.inverse_transform(X_test_tf[i, :])[0], w2v_model.vector_size, X_test_tf[i,:].data, False)
w2v_X_test.shape

(124, 100)

In [19]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks

# # Create the SMOTE object
# sm = SMOTE(sampling_strategy=1, k_neighbors=5, random_state=10)

# # Apply SMOTE to the training data
# X_train_resampled, y_train_resampled = sm.fit_resample(w2v_X_train, Y_train)

# print('Resampled dataset shape %s' % Counter(y_train_resampled))

undersampler_nn = TomekLinks(sampling_strategy="all") # use tomeLinks to remove ambiguous data through nearest neighbours


# Fit and transform the X and y data
X_train_resampled, y_train_resampled = undersampler_nn.fit_resample(w2v_X_train, Y_train)

# X_train_resampled, y_train_resampled = w2v_X_train, Y_train


print('Resampled dataset shape %s' % Counter(y_train_resampled))

Resampled dataset shape Counter({1: 319, 0: 231})


In [20]:
from keras.layers import Flatten

In [21]:
from keras.utils import to_categorical
Y_train_hot = to_categorical(y_train_resampled)

In [22]:
Y_test_hot = to_categorical(Y_test)

In [23]:
from keras.callbacks import EarlyStopping

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='binary_accuracy', patience=25)

# Fit the model with early stopping
model = Sequential()
model.add(Dense(300, activation='relu', input_dim=w2v_model.vector_size))
model.add(Dropout(0.2))
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

# Compile your model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

In [24]:
model.fit(X_train_resampled, Y_train_hot, epochs=100, batch_size=32, verbose=2, validation_split=0.1, callbacks = [early_stopping])

Epoch 1/100
16/16 - 3s - loss: 0.6503 - binary_accuracy: 0.6232 - val_loss: 0.9424 - val_binary_accuracy: 0.0000e+00 - 3s/epoch - 195ms/step
Epoch 2/100
16/16 - 0s - loss: 0.5236 - binary_accuracy: 0.6838 - val_loss: 0.5761 - val_binary_accuracy: 0.8545 - 154ms/epoch - 10ms/step
Epoch 3/100
16/16 - 0s - loss: 0.3603 - binary_accuracy: 0.8727 - val_loss: 0.2304 - val_binary_accuracy: 0.9636 - 158ms/epoch - 10ms/step
Epoch 4/100
16/16 - 0s - loss: 0.2638 - binary_accuracy: 0.8949 - val_loss: 0.0914 - val_binary_accuracy: 0.9636 - 155ms/epoch - 10ms/step
Epoch 5/100
16/16 - 0s - loss: 0.2207 - binary_accuracy: 0.9111 - val_loss: 0.1098 - val_binary_accuracy: 0.9636 - 143ms/epoch - 9ms/step
Epoch 6/100
16/16 - 0s - loss: 0.2054 - binary_accuracy: 0.9192 - val_loss: 0.0894 - val_binary_accuracy: 0.9636 - 147ms/epoch - 9ms/step
Epoch 7/100
16/16 - 0s - loss: 0.1479 - binary_accuracy: 0.9394 - val_loss: 0.0677 - val_binary_accuracy: 0.9636 - 146ms/epoch - 9ms/step
Epoch 8/100
16/16 - 0s - los

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               30300     
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 300)               90300     
                                                                 
 dropout_1 (Dropout)         (None, 300)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               30100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 2

In [26]:
predictions = model.predict(w2v_X_test) 

4/4 [==============================] - 0s 3ms/step


In [27]:
predictions = np.round(predictions)

In [28]:
from sklearn.metrics import classification_report
print(classification_report(Y_test_hot, predictions))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75        41
           1       0.88      0.87      0.87        83

   micro avg       0.83      0.83      0.83       124
   macro avg       0.81      0.81      0.81       124
weighted avg       0.83      0.83      0.83       124
 samples avg       0.83      0.83      0.83       124

